# signac-flow minimal example

## About

This notebook contains a minimal example for running a signac-flow project from scratch.
The example demonstrates how to compare an ideal gas with a lennard jones fluid by calculating a p-V phase diagram.

## Author

Carl Simon Adorf

## Before you start

Make sure you installed signac and signac-flow, e.g., with:

```
conda install -c glotzer  signac
conda install -c glotzer  signac-flow
```

In [ ]:
import signac
import flow
import numpy as np


# Enter the signac project directory
project = signac.init_project('FlowTutorialProject', '../../projects/tutorial-signac-flow')

We want to generate a pressure-volume (p-V) phase diagram for an ideal gas.

We define a function to calculate the result for a given state point:

In [ ]:
def V_idg(N, p, kT):
    return N * kT / p

We want to use **signac** to manage our data, therefore we define an *operation* which has only the *signac job* as argument:

In [ ]:
def compute_volume(job):
    job.document['V'] = V_idg(** job.statepoint())

For this demonstration we will specialize a `flow.FlowProejct` to manage our simple *workflow*.

The workflow is controlled by two core functions: `classify()` and `next_operation()`:
 - The `classify()` function allows us to *label* our jobs and get a good overview of the project *status*. This is especially important once the data space becomes larger and more complex and operations more expensive.
 - The `next_operation()` functions helps to automate the workflow by identifying the next required operation for each job.
 
 In this case there is only **one** operation:

In [ ]:
class MyProject(flow.FlowProject):
    
    def classify(self, job):
        yield 'init'
        if 'V' in job.document:
            yield 'estimated'
    
    def next_operation(self, job):
        labels = set(self.classify(job))
        if 'V' not in job.document:
            return 'compute_volume'

We need to use the `get_project()` *class method* to get a project handle for this special project class.

In [ ]:
project = MyProject.get_project(root='../../projects/tutorial-signac-flow')

Now it's time to actually generate some data! Let's initialize the data space!


In [ ]:
for p in np.linspace(0.5, 5.0, 10):
    sp = dict(N=1728, kT=1.0, p=p)
    project.open_job(sp).init()

The `print_status()` function allows to get a quick overview of our project's *status*:

In [ ]:
project.print_status(detailed=True, parameters=['p'])

The next cell will attempt to execute all operations by cycling through jobs and operations until no *next operations* are defined anymore.

We limit the max. number of cycles to prevent accidental infinite loops, the number of cycles is arbitrary.

In [ ]:
for i in range(3):
    for job in project.find_jobs():
        for j in range(5):
            next_op = project.next_operation(job)
            if next_op is None:
                break
            print('execute', job, next_op)
            globals()[next_op](job)
            assert next_op != project.next_operation(job)
        else:
            raise RuntimeError("Reached max. # cycle limit!")

Let's double check the project status.

In [ ]:
project.print_status()

After running all operations we can make a brief examination of the collected data.

In [ ]:
for job in project.find_jobs():
    print(job.statepoint()['p'], job.document.get('V'))

For a better presentation of the results we need to aggregate all results and sort them by pressure.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

V = dict()

for job in project.find_jobs():
    V[job.statepoint()['p']] = job.document['V']
    
p = sorted(V.keys())
V = [V[p_] for p_ in p]
print(V)

plt.plot(p, V, label='idG')
plt.xlabel(r'pressure [$\epsilon / \sigma^3$]')
plt.ylabel(r'volume [$\sigma^3$]')
plt.legend()

As a a final step, we ca generate a index of our project data.
You can store this index in a variable or within a database, e.g., for search operations.

In [ ]:
for doc in project.index():
    print(doc)

Uncomment and execute the following line to remove all data and start over.

In [ ]:
#% rm -r projects/tutorial-signac-flow/workspace